# Many-Body Expansion with libfrag

This notebook demonstrates the Many-Body Expansion (MBE) functionality in libfrag for fragment-based quantum chemistry calculations.

In [ ]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
import libfrag
from libfrag.mbe import *

## 1. Creating a Molecular System

Let's start by creating a water dimer system to demonstrate MBE calculations.

In [ ]:
# Create water dimer atoms
atoms = [
    # Water molecule 1
    libfrag.Atom("O", 0.000,  0.000,  0.000),
    libfrag.Atom("H", 0.757,  0.587,  0.000),
    libfrag.Atom("H", -0.757, 0.587,  0.000),
    
    # Water molecule 2 (3 Å away)
    libfrag.Atom("O", 3.000,  0.000,  0.000),
    libfrag.Atom("H", 3.757,  0.587,  0.000),
    libfrag.Atom("H", 2.243,  0.587,  0.000)
]

molecule = libfrag.Molecule(atoms)
print(f"Created water dimer with {len(molecule)} atoms")
print(f"Molecular formula: {molecule.formula()}")

## 2. Configuring MBE Calculations

Configure the MBE calculation with different settings and fragmentation schemes.

In [ ]:
# Create basic 2-body MBE configuration
config_2body = MBEConfig.default_2body()
config_2body.set_qm_method("HF")
config_2body.set_basis_set("6-31G*")
config_2body.set_fragmentation_scheme(FragmentationScheme.MOLECULAR)

print("2-body MBE Configuration:")
print(config_2body.to_string())

In [ ]:
# Create 3-body configuration for comparison
config_3body = MBEConfig.default_3body()
config_3body.set_qm_method("B3LYP")
config_3body.set_basis_set("def2-TZVP")
config_3body.set_distance_cutoff(5.0)  # Only consider fragments within 5 Å

print("3-body MBE Configuration:")
print(config_3body.to_string())

## 3. Fragment Generation Analysis

Analyze how many fragment combinations will be generated for different configurations.

In [ ]:
# Create fragment generator
generator = MBEFragmentGenerator(config_2body)

# Estimate combinations for different system sizes
fragment_counts = []
orders = range(1, 5)
n_fragments_list = [2, 5, 10, 20]

print("Estimated number of fragment combinations:")
print(f"{'N-fragments':<12} {'1-body':<8} {'2-body':<8} {'3-body':<8} {'4-body':<8}")
print("-" * 50)

for n_fragments in n_fragments_list:
    row = [str(n_fragments)]
    for order in orders:
        n_combinations = generator.estimate_combinations(n_fragments, order)
        row.append(str(n_combinations))
    print(f"{row[0]:<12} {row[1]:<8} {row[2]:<8} {row[3]:<8} {row[4]:<8}")

In [ ]:
# Visualize scaling of combinations
n_fragments_range = range(2, 21)
combinations_2body = [generator.estimate_combinations(n, 2) for n in n_fragments_range]
combinations_3body = [generator.estimate_combinations(n, 3) for n in n_fragments_range]

plt.figure(figsize=(10, 6))
plt.semilogy(n_fragments_range, combinations_2body, 'o-', label='2-body', linewidth=2)
plt.semilogy(n_fragments_range, combinations_3body, 's-', label='3-body', linewidth=2)
plt.xlabel('Number of Fragments')
plt.ylabel('Number of Combinations (log scale)')
plt.title('MBE Computational Scaling')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Mock MBE Results Analysis

Since we don't have a QM calculator connected, let's create mock results to demonstrate the analysis capabilities.

In [ ]:
# Create MBE results container
results = MBEResults(config_2body)

# Add mock 1-body results (individual water molecules)
for i in range(2):
    fragment_result = FragmentCalculationResult([i], 1)
    fragment_result.fragment_id = f"water_{i+1}"
    fragment_result.total_energy = -76.2 + np.random.normal(0, 0.001)  # Mock HF/6-31G* water energy
    fragment_result.qm_method = "HF"
    fragment_result.basis_set = "6-31G*"
    fragment_result.converged = True
    fragment_result.computation_time = np.random.uniform(5, 15)  # seconds
    results.add_fragment_result(fragment_result)

# Add mock 2-body result (water dimer interaction)
dimer_result = FragmentCalculationResult([0, 1], 2)
dimer_result.fragment_id = "water_dimer"
dimer_result.total_energy = -152.41  # Total dimer energy
dimer_result.qm_method = "HF"
dimer_result.basis_set = "6-31G*"
dimer_result.converged = True
dimer_result.computation_time = 45.0  # seconds
results.add_fragment_result(dimer_result)

print(f"Added {results.n_calculations()} fragment calculations")
print(f"Total MBE energy: {results.total_energy():.6f} Hartree")

In [ ]:
# Analyze energy contributions
print("Energy Decomposition:")
print(results.energy_decomposition_table())

# Calculate interaction energy
e_1body_total = sum(results.fragment_energies())
e_2body_total = results.energy_contribution(2)
interaction_energy = e_2body_total - e_1body_total

print(f"\nInteraction Analysis:")
print(f"Sum of monomers: {e_1body_total:.6f} Hartree")
print(f"Dimer total:     {e_2body_total:.6f} Hartree")
print(f"Interaction:     {interaction_energy:.6f} Hartree")
print(f"Interaction:     {interaction_energy * 627.5:.2f} kcal/mol")

In [ ]:
# Visualize energy contributions
contributions = results.energy_contributions()
orders = list(contributions.keys())
energies = list(contributions.values())

plt.figure(figsize=(8, 6))
plt.bar(orders, energies, alpha=0.7, color=['blue', 'orange'])
plt.xlabel('N-body Order')
plt.ylabel('Energy Contribution (Hartree)')
plt.title('MBE Energy Decomposition')
plt.xticks(orders)
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, (order, energy) in enumerate(zip(orders, energies)):
    plt.text(order, energy + 0.5, f'{energy:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 5. Performance Analysis

Analyze computational performance and timing.

In [ ]:
# Performance statistics
perf_stats = results.performance_statistics()

print("Performance Statistics:")
for metric, value in perf_stats.items():
    if 'time' in metric:
        print(f"{metric}: {value:.2f} seconds")
    else:
        print(f"{metric}: {value}")

In [ ]:
# Timing breakdown by order
orders = range(1, results.max_order() + 1)
times_by_order = [results.computation_time_by_order(order) for order in orders]

plt.figure(figsize=(8, 5))
plt.bar(orders, times_by_order, alpha=0.7, color='green')
plt.xlabel('N-body Order')
plt.ylabel('Computation Time (seconds)')
plt.title('Computation Time by N-body Order')
plt.xticks(orders)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Export and Reporting

Demonstrate various export formats and reporting capabilities.

In [ ]:
# Generate comprehensive summary report
print("=== MBE Calculation Summary ===")
print(results.summary_report())

In [ ]:
# JSON export (formatted for display)
import json

json_data = json.loads(results.to_json())
print("JSON Export:")
print(json.dumps(json_data, indent=2))

In [ ]:
# CSV export for spreadsheet analysis
csv_data = results.to_csv()
print("CSV Export:")
print(csv_data)

# Save to file
with open('mbe_results.csv', 'w') as f:
    f.write(csv_data)
print("\nResults saved to 'mbe_results.csv'")

## 7. Advanced Configuration Examples

Demonstrate advanced configuration options and custom fragmentation.

In [ ]:
# Custom fragmentation scheme
custom_config = MBEConfig(3, FragmentationScheme.CUSTOM)

# Define fragments manually (atom indices)
custom_fragments = [
    [0, 1, 2],  # First water molecule
    [3, 4, 5]   # Second water molecule
]

custom_config.set_custom_fragments(custom_fragments)
custom_config.set_qm_method("B3LYP")
custom_config.set_basis_set("def2-TZVP")
custom_config.set_distance_cutoff(6.0)
custom_config.set_energy_threshold(1e-5)
custom_config.set_charge_embedding(True)

# Add QM software options
qm_options = {
    "scf_convergence": "1e-8",
    "grid": "ultrafine",
    "integral_threshold": "1e-12"
}
custom_config.set_qm_options(qm_options)

print("Custom Configuration:")
print(custom_config.to_string())

In [ ]:
# Compare different fragmentation schemes
schemes = [
    (FragmentationScheme.ATOMIC, "Atomic"),
    (FragmentationScheme.MOLECULAR, "Molecular"),
    (FragmentationScheme.DISTANCE_BASED, "Distance-based"),
    (FragmentationScheme.CUSTOM, "Custom")
]

print("Fragmentation Scheme Comparison:")
print(f"{'Scheme':<15} {'Description':<30} {'Best for'}")
print("-" * 70)

descriptions = {
    "Atomic": "Each atom is a fragment",
    "Molecular": "Connected components",
    "Distance-based": "Spatial clustering",
    "Custom": "User-defined fragments"
}

best_for = {
    "Atomic": "Detailed analysis, small systems",
    "Molecular": "Intermolecular interactions", 
    "Distance-based": "Large systems, efficiency",
    "Custom": "Expert knowledge, special cases"
}

for scheme, name in schemes:
    print(f"{name:<15} {descriptions[name]:<30} {best_for[name]}")

## 8. Utility Functions

Demonstrate utility functions for combinatorics and fragment analysis.

In [ ]:
# Combinatorial utilities
print("Combinatorial Utilities:")

# Binomial coefficients
for n in [5, 10, 20]:
    for k in range(1, min(5, n+1)):
        coeff = binomial_coefficient(n, k)
        print(f"C({n},{k}) = {coeff}")
    print()

In [ ]:
# Generate all combinations
print("All 2-combinations from 5 items:")
combinations = generate_combinations(5, 2)
for i, combo in enumerate(combinations):
    print(f"{i+1:2d}: {combo}")

print(f"\nTotal: {len(combinations)} combinations")

In [ ]:
# Test combination overlap
test_combinations = [
    ([0, 1, 2], [2, 3, 4]),  # Overlap at index 2
    ([0, 1], [2, 3]),        # No overlap
    ([1, 3, 5], [0, 2, 4])   # No overlap
]

print("Combination Overlap Testing:")
for combo1, combo2 in test_combinations:
    overlap = combinations_overlap(combo1, combo2)
    print(f"{combo1} ∩ {combo2} = {overlap}")

## Summary

This notebook has demonstrated the key features of the libfrag MBE module:

1. **Configuration Management**: Flexible setup of MBE calculations with multiple fragmentation schemes
2. **Results Analysis**: Comprehensive energy decomposition and convergence analysis
3. **Performance Monitoring**: Timing analysis and computational cost estimation
4. **Export Capabilities**: Multiple output formats (JSON, CSV, reports)
5. **Utility Functions**: Combinatorial tools and fragment manipulation

The MBE module provides a complete framework for fragment-based quantum chemistry calculations, with hooks for integration with various QM software packages.

**Next Steps**:
- Implement QM calculator interfaces for specific software (PySCF, Gaussian, ORCA, etc.)
- Add real molecular fragmentation algorithms
- Optimize performance for large systems
- Add visualization capabilities
- Implement gradient and property calculations